In [1]:
!pip install imageio

    100% |████████████████████████████████| 3.3MB 3.3MB/s ta 0:00:011    55% |█████████████████▉              | 1.8MB 9.4MB/s eta 0:00:01


In [29]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd
import imageio
import PIL
import os
import sys
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(train_images,train_labels),(_,__)=tf.keras.datasets.mnist.load_data()

In [4]:
train_images=train_images.reshape(train_images.shape[0],28,28,1).astype(np.float32)
train_images = (train_images - 127.5)/127.5

In [5]:
BUFFER_SIZE=60000
BATCH_SIZE = 256

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(True).batch(BATCH_SIZE)

In [18]:
train_images.shape

(60000, 28, 28, 1)

In [9]:
def make_generative_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(7*7*256,use_bias=False,input_shape=(100,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Reshape((7,7,256)))
    assert model.output_shape == (None,7,7,256)
    model.add(tf.keras.layers.Conv2DTranspose(filters=128,kernel_size=(5,5),strides=1,padding="same",use_bias=False))
    assert model.output_shape == (None,7,7,128)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding="same",use_bias=False))
    assert model.output_shape == (None,14,14,64)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(1,(5,5),strides=2,padding="same",use_bias=False,activation='tanh'))
    assert model.output_shape == (None,28,28,1)
    return model

In [10]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64,(5,5),strides=(2,2),padding="same"))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(128,(5,5),strides=(2,2),padding="same"))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    return model
    

In [11]:
generator=make_generative_model()
discriminator = discriminator_model()

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
def generator_loss(generated_output):
   return tf.losses.sigmoid_cross_entropy(tf.ones_like(generated_output),generated_output)

In [15]:
def discriminator_loss(real_output,generated_output):
    real_loss=tf.losses.sigmoid_cross_entropy(tf.ones_like(real_output),real_output)
    generated_loss=tf.losses.sigmoid_cross_entropy(tf.zeros_like(generated_output),generated_output)
    total_loss = real_loss + generated_loss
    return total_loss

In [16]:
generator_optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

In [22]:
checkpoint_dir='./training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')
checkpoint= tf.train.Checkpoint(generator_optimizer=generator_optimizer,discriminator_optimizer=discriminator_optimizer,generator=generator,discriminator=discriminator)

In [24]:
EPOCHS = 50
noise_dim = 100
num_ex_to_generate=16
random_vector_for_generation = tf.random_normal([num_ex_to_generate,noise_dim])

In [34]:
def train_step(images):
    noise = tf.random_normal([BATCH_SIZE,noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as desc_tape:
        generated_images = generator(noise,training=True)
        discriminator_image = discriminator(images,training=True)
        generated_output = discriminator(generated_images,training=True)
        gen_loss = generator_loss(generated_output)
        disc_loss = discriminator_loss(discriminator_image,generated_output)
    gradient_of_generator = gen_tape.gradient(gen_loss,generator.variables)
    gradient_of_disc = desc_tape.gradient(disc_loss,discriminator.variables)
    generator_optimizer.apply_gradients(zip(gradient_of_generator,generator.variables))
    discriminator_optimizer.apply_gradients(zip(gradient_of_disc,discriminator.variables))

In [35]:
train_step = tf.contrib.eager.defun(train_step)

In [36]:
def train(images,epochs):
    for epoch in range(epochs):
        start = time.time()
        for img in images:
            train_step(img)
        display.clear_output(wait=True)
        generate_and_save_images(generator,epoch+1,random_vector_for_generation)
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_preyfix=checkpoint_prefix)
        print("Time taken for epoch {} is {} sec".format(epoch,time.time()-start))
    display.clear_output(wait=True)
    generate_and_save_images(generator,epochs,random_vector_for_generation)

In [37]:
def generate_and_save_images(model,epoch,test_input):
    prediction = model(test_input,training=False)
    fig = plt.figure(figsize=(4,4))
    for i in range(prediction.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(prediction[i,:,:,0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
train(train_dataset,EPOCHS)